In [ ]:
import os
import yt_dlp
import cv2
from ultralytics import YOLO
from moviepy.editor import ImageSequenceClip

### 1. Yutube 영상 다운로드

In [ ]:
# 다운로드할 YouTube URL
url = ""

# yt-dlp 설정
ydl_opts = {
    "format": "best",
    "outtmpl": "downloaded_video.%(ext)s",
}

# 다운로드
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])

### 2. 동영상의 fps 정보

In [ ]:
# 동영상 파일 경로
video_path = "downloaded_video.mp4"

# 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# FPS 가져오기
fps = cap.get(cv2.CAP_PROP_FPS)
print("FPS:", fps)

# 동영상 파일 닫기
cap.release()

### 3. 동영상을 프레임 단위로 분할

In [ ]:
video_path = "downloaded_video.mp4"

# cv2를 이용해 동영상 파일 열기
cap = cv2.VideoCapture(video_path)

# 프레임을 저장할 디렉토리 생성
output_dir = "frames"
os.makedirs(output_dir, exist_ok=True)

# 프레임 단위로 이미지로 저장
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    # 이미지 파일로 프레임 저장
    frame_path = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
    cv2.imwrite(frame_path, frame)
    frame_count += 1

# 동영상 파일 닫기
cap.release()
print(f"총 {frame_count}개의 프레임이 저장되었습니다.")

### 4. YOLOv8 추론

In [ ]:
# Load a model
model = YOLO("yolov8n.pt")  # pretrained YOLO8n model

# "frames" 디렉토리 안에 있는 파일과 폴더 목록을 가져옴
frames = os.listdir("frames/")

# 원하는 디렉토리 경로 설정
directory_path = "results"

# 디렉토리가 없으면 생성
if not os.path.exists(directory_path):
    os.makedirs(directory_path)
    print(f"디렉토리 '{directory_path}'가 생성되었어.")
else:
    print(f"디렉토리 '{directory_path}'가 이미 존재해.")

for frame in frames:
    # YOLOv8 모델을 사용하여 객체 감지 수행
    results = model("frames/" + frame)  # , device="mps" (맥북)

    # Process results list
    for result in results:
        result.save(filename="results/" + frame)  # save to disk

### 5. 프레임을 동영상으로 변환 - moviepy 라이브러리

In [ ]:
# 이미지 파일들이 저장된 디렉토리 경로
image_folder = "results"  # 이미지 파일들이 있는 폴더 경로
fps = int(fps)  # 초당 프레임 수 설정

# 이미지 파일 리스트 가져오기 (예: jpg, png 형식)
image_files = [
    os.path.join(image_folder, img)
    for img in sorted(os.listdir(image_folder))
    if img.endswith((".jpg", ".png"))
]

# 이미지 시퀀스를 이용해 클립 생성
clip = ImageSequenceClip(image_files, fps=fps)

# 동영상 파일로 저장
output_path = "output_video.mp4"
clip.write_videofile(output_path, codec="libx264")

print(f"동영상이 생성되었습니다: {output_path}")